<a href="https://www.kaggle.com/code/samithhegde/spaceship-titanic-competition?scriptVersionId=129068578" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
train.head()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train["VIP"].value_counts()

In [ ]:
train1 = train.fillna(train.mode().iloc[0])
train1.head()

In [ ]:
train1["VIP"].value_counts()

In [ ]:
train["HomePlanet"].value_counts()

In [ ]:
train1["HomePlanet"].value_counts()

In [ ]:
train["CryoSleep"].value_counts()

In [ ]:
train1["CryoSleep"].value_counts()

In [ ]:
train["Destination"].value_counts()

In [ ]:
train["Transported"].value_counts()

In [ ]:
train["Age"].mean()

In [ ]:
train1["Age"].mean()

In [ ]:
train[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]].mean()

In [ ]:
train1[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]].mean()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train1.hist(bins=50, figsize=(12,7))

In [ ]:
sns.countplot(data=train1, x='HomePlanet', hue='Transported')

In [ ]:
sns.countplot(data=train1, x='CryoSleep', hue='Transported')

In [ ]:
sns.countplot(data=train1, x='Destination', hue='Transported')

In [ ]:
sns.countplot(data=train1, x='VIP', hue='Transported')

In [ ]:
plt.figure(figsize=(12,7))
sns.heatmap(train1.corr(), annot=True, cmap='crest')

In [ ]:
y = train1["Transported"]
x = train1.drop("Transported", axis=1)
x.head()

In [ ]:
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test.head()

In [ ]:
test.isnull().sum()

In [ ]:
test1 = test.fillna(test.mode().iloc[0])

In [ ]:
num_feats = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
cat_feats = ["HomePlanet", "CryoSleep", "Destination", "VIP"]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

num_pipe = Pipeline([
    ('scaler', StandardScaler())
])

In [ ]:
num_list = list(num_feats)
cat_list = list(cat_feats)

In [ ]:
final_pipe = ColumnTransformer([
    ('num', num_pipe, num_list),
    ('cat', OneHotEncoder(), cat_list)
])

In [ ]:
x_train = final_pipe.fit_transform(x)

In [ ]:
x_train

In [ ]:
#from sklearn.svm import SVC
#svc = SVC(kernel='poly')
#svc.fit(x_train, y)

In [ ]:
#from sklearn.metrics import accuracy_score
#svc_pred = svc.predict(x_train)
#accuracy_score(y, svc_pred)

In [ ]:
#from sklearn.model_selection import GridSearchCV

#params_grid = {
 #   'C':[10,100],
 #   'gamma':[0.1,0.01,0.001],
 #   'degree':[2,3,4]
#}

#grid_search = GridSearchCV(SVC(kernel='poly'), params_grid, verbose=2)
#grid_search.fit(x_train, y)

In [ ]:
#grid_search.best_params_

In [ ]:
#poly_best = grid_search.best_estimator_
#grid_pred = poly_best.predict(x_train)

In [ ]:
#accuracy_score(y, grid_pred)

In [ ]:
x_test = final_pipe.transform(test1)
x_test

In [ ]:
x.isnull().sum()

In [ ]:
test1.isnull().sum()

In [ ]:
len(x_train)

In [ ]:
len(x_test)

In [ ]:
#final_prediction = poly_best.predict(x_test)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag = BaggingClassifier(
    DecisionTreeClassifier(class_weight='balanced'),
    max_samples=0.5, bootstrap=False, n_estimators=300, max_features=0.5
)

bag.fit(x_train, y)

In [ ]:
from sklearn.metrics import accuracy_score

pred_bag = bag.predict(x_train)
accuracy_score(y, pred_bag)

In [ ]:
pred_bag_final = bag.predict(x_test)

In [ ]:
output = pd.DataFrame({'PassengerId': test1.PassengerId, 'Transported':pred_bag_final})
output.to_csv('submission.csv', index=False)